In [ ]:
from keras.utils import Sequence
from google.colab import drive
from glob import glob
import numpy as np
import matplotlib.image as img
from skimage.transform import resize
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

In [ ]:
drive.mount('/content/drive')

In [ ]:
class GeradorSequencias(Sequence):

    def __init__(self, entradas, saidas, batch_size):
        self.entradas = entradas
        self.saidas = saidas
        self.batch_size  = batch_size

    #como calcular a quantidade de lotes: (tamanho do dataset / tamanho do lote)
    def __len__(self):
        return len(self.entradas) // (self.batch_size) # "//" pega o inteiro

    #forma de acessar o índice do lote ([0, 1]) --> para obter o índice 0, multiplique o nº do lote pelo seu tamanho
    def __getitem__(self, id):
        esquerda = id * self.batch_size
        direita = min(esquerda + self.batch_size, len(self.entradas))
        batch_entradas = self.entradas[esquerda:direita]
        batch_saidas = self.saidas[esquerda:direita]

        #redimensionar as imagens de entrada (já está normalizada)
        entradas_redimensionadas = []
        for caminho_img in batch_entradas:
            img_entrada = img.imread(caminho_img + 'image.tif')
            img_redimensionada_entrada = resize(img_entrada, (256, 256))
            entradas_redimensionadas.append(img_entrada)

        #redimensionar as máscaras de saída
        saidas_redimensionadas = []
        for caminho_img in batch_saidas:
            img_saida = img.imread(caminho_img + 'mask.png')
            img_redimensionada_saida = resize(img_saida, (256, 256))
            img_redimensionada_saida = np.array(img_redimensionada_saida)
            min_val = np.min(img_redimensionada_saida)
            max_val = np.max(img_redimensionada_saida)
            img_redimensionada_saida = (img_redimensionada_saida - min_val) / (max_val - min_val)
            threshold = 0.5
            img_redimensionada_saida = tf.where(img_redimensionada_saida < threshold, 0.0, 1.0)
            saidas_redimensionadas.append(img_redimensionada_saida)

        return np.array(entradas_redimensionadas), np.array(saidas_redimensionadas)

In [ ]:
caminhos = glob('/content/drive/Shareddrives/Grupo T de Tech/Data/dataset_inteli/cropped_images/*/*/')
np.random.shuffle(caminhos)

In [ ]:
sequencia = GeradorSequencias(caminhos, caminhos, 32)

In [ ]:
primeiro_lote_entradas, primeiro_lote_saidas = sequencia.__getitem__(0)

In [ ]:
img_entrada = primeiro_lote_entradas[9]

# Mostrando as imagens
plt.figure(figsize=(10, 8))
plt.subplot(1, 2, 1)
plt.imshow(img_entrada, cmap='gray')
plt.title('Entrada')
plt.axis('off')


plt.show()

In [ ]:
#image_equalized = cv2.equalizeHist(img_entrada)
image_gray = cv2.cvtColor(img_entrada, cv2.COLOR_BGR2GRAY)
image_equalized = cv2.equalizeHist(image_gray)

In [ ]:
plt.figure(figsize=(10, 8))
plt.subplot(1, 2, 1)
plt.imshow(image_equalized, cmap="gray")
plt.title('Entrada')
plt.axis('off')


plt.show()

In [ ]:
# Compute Sobel gradients
sobelX = cv2.Sobel(image_equalized, cv2.CV_64F, 1, 0, ksize=3)
sobelY = cv2.Sobel(image_equalized, cv2.CV_64F, 0, 1, ksize=3)

# Compute the gradient magnitude
sobelCombined = cv2.magnitude(sobelX, sobelY)

# Normalizar a imagem resultante para melhorar a visualização
sobelCombined_normalized = cv2.normalize(sobelCombined, None, 0, 255, cv2.NORM_MINMAX)

# Mostrando as imagens
plt.figure(figsize=(10, 8))
plt.subplot(1, 2, 1)
plt.imshow(sobelCombined_normalized, cmap='gray')
plt.axis('off')


plt.show()